In [1]:
import os
import pandas as pd

os.chdir("../../")
from scripts.llm import get_num_of_tokens, get_completion
from scripts.utils import save_obj_as_pickle, read_obj_from_pickle
from scripts.data import make_database, make_prompts_for_clf

pd.set_option("display.max_colwidth", 150)

In [2]:
def loadData(fp, sample_size=1000):
    df = pd.read_csv(fp)
    df.rename(columns={"Class Index": "label", "Description": "text"}, inplace=True)
    df.label = df.label.map({1: "World", 2: "Sports", 3: "Business", 4: "Sci/Tech"})
    df = df[df.text.apply(get_num_of_tokens) <= 40].sample(sample_size, random_state=234)
    df = df[["text", "label"]].reset_index(drop=True)
    return df

clf_task = "AGNews"
test_fp = f"data/raw/text classification/{clf_task}/test.csv"
test_df = loadData(test_fp)

dev_fp = f"data/raw/text classification/{clf_task}/train.csv"
dev_df = loadData(dev_fp)

test_df.head()

,text,label
0,The New York Jets and quarterback Chad Pennington are looking to finalize a contract extension by next Wednesday.,Sports
1,TEHRAN: Iran added one more missile to its military arsenal and the defense minister said Saturday his country was ready to confront any external ...,World
2,AFP - The second major airlift of Vietnamese Montagnards who fled to Cambodia's remote jungles after April anti-government protests will begin at ...,World
3,"Out of money, out of patience, out of time, and for the foreseeable future, out of business.",Sports
4,"NEW YORK, September 3 (New Ratings) - The European Union has reportedly made significant progress in settling its prolonged antitrust case against...",Business


In [ ]:
(test_df.label.value_counts() / len(test_df)).round(4) * 100

In [ ]:
len(test_df)

In [4]:
test_df.label.value_counts()

World       261
Sci/Tech    254
Business    243
Sports      242
Name: label, dtype: int64

In [5]:
test_df.text.apply(get_num_of_tokens).describe()

count    1000.000000
mean       31.660000
std         6.423057
min         8.000000
25%        28.000000
50%        33.000000
75%        37.000000
max        40.000000
Name: text, dtype: float64

In [6]:
dev_df.label.value_counts()

Sci/Tech    275
World       252
Sports      247
Business    226
Name: label, dtype: int64

In [7]:
dev_df.text.apply(get_num_of_tokens).describe()

count    1000.0000
mean       31.6450
std         6.1985
min         9.0000
25%        28.0000
50%        33.0000
75%        37.0000
max        40.0000
Name: text, dtype: float64

### Database

Sources for the prompt data

In [40]:
database = make_database(test_df, dev_df, num_instance=500, max_instance_size=500)
database.keys()

dict_keys(['num_instance', 'max_instance_size', 'labels', 'testData', 'testInstances', 'devData', 'devInstances'])

#### Zero-shot

In [43]:
database["promptTemplates"] = dict()
database["promptTemplates"]["zero-shot"] = dict()

single_clf = "Classify which news category the following line of text belongs to among the following four categories: " \
             "'Business,' 'Sports,' 'World,' and 'Sci/Tech.'\n\n" \
             "Text: $text\nNews category:"

batch_clf = "Classify which news category each of the $num following lines of text belongs to " \
            "among the following four categories: 'Business,' 'Sports,' 'World,' and 'Sci/Tech.'\n\n" \
            "Texts, one per line:\n\n$texts\n\n" \
            "News categories for each of the $num lines of text, one per line:\n"

index_selection_one_cat_a_time = "This is a news classification task in which each line of text belongs to one of four categories 'Business,' 'Sports,' 'World,' and 'Sci/Tech.'\n\n" \
                                 "Go over the $num lines of text below and list the index numbers of the lines that can be classified as $category according to the following instructions:\n" \
                                 "If none of the texts can be classified as $category, write 'None.'\n" \
                                 "If all the texts can be classified as $category, write 'All.'\n" \
                                 "Otherwise, provide the index numbers of the texts that can be classified as $category, each on a separate line.\n\n" \
                                 "Texts, one per line:\n\n$texts\n\n" \
                                 "'None,' 'All,' or the index numbers of the texts that can be classified as $category, one per line:\n"

index_selection_one_cat_a_time_json =   "This is a news classification task in which each line of text belongs to one of four categories 'Business,' 'Sports,' 'World,' and 'Sci/Tech.'\n\n" \
                                        "Go over the $num lines of text below and list the index numbers of the lines that can be classified as $category according to the following instructions:\n" \
                                        "If none of the texts can be classified as $category, write 'None.'\n" \
                                        "If all the texts can be classified as $category, write 'All.'\n" \
                                        "Otherwise, provide the index numbers of the texts that can be classified as $category.\n\n" \
                                        "Output your responses in JSON format with the key '$category'.\nA formatted example output is provided below.\n" \
                                        "{'$category': [None/All or index numbers of the texts that can be classified as $category]}\n\n" \
                                        "Texts, one per line:\n\n$texts\n\n" \
                                        "JSON output:\n"

index_selection_all_cat_at_once = "This is a news classification task in which each line of text belongs to one of four categories 'Business,' 'Sports,' 'World,' and 'Sci/Tech.'\n\n" \
                                  "Go over the $num lines of text below and list the index numbers of the lines that belong to each category according to the following instructions:\n" \
                                  "If none of the texts can be classified as a particular category, write 'None.'\n" \
                                  "If all the texts can be classified as a particular category, write 'All.'\n" \
                                  "Otherwise, provide the index numbers of the texts that can be classified as the category.\n" \
                                  "Output your responses in JSON format with the following keys: 'business,' 'sports,' 'world,' and 'sci/tech.'\n" \
                                  "A formatted example output is provided below.\n" \
                                  "{'business': [None/All or index numbers of texts in 'business' category], 'sports': [None/All or index numbers of texts in 'sports' category], " \
                                  "'world': [None/All or index numbers of texts in 'world' category], 'sci/tech': [None/All or index numbers of texts in sci/tech category]}\n\n" \
                                  "Texts, one per line:\n\n$texts\n\n" \
                                  "JSON output:\n" 

tasks = ["single_clf", "batch_clf", "index_selection_one_cat_a_time", "index_selection_all_cat_at_once", "index_selection_one_cat_a_time_json"]
promptTemplates = [single_clf, batch_clf, index_selection_one_cat_a_time, index_selection_all_cat_at_once, index_selection_one_cat_a_time_json]

for task, tmp in zip(tasks, promptTemplates):
    database["promptTemplates"]["zero-shot"][task] = tmp

In [44]:
os.makedirs("data/databases/text classification/", exist_ok=True)
save_obj_as_pickle(database, f"data/databases/text classification/{clf_task}.pkl")

Saved object to data/databases/text classification/AGNews.pkl


#### Test Prompts

- The main purpose is to check if LLMs can output the desired formats given the prompts 

In [11]:
dev = []
num_instance = 2

taskSizes = [5, 10]
for propmtMode in ["zero-shot"]:
    for task in tasks:
        if task == "single_clf":
            dev.append(make_prompts_for_clf(database, task, "dev", propmtMode)[:num_instance])
            continue

        for taskSize in taskSizes:  
            dev.append(make_prompts_for_clf(database, task, "dev", propmtMode, taskSize, attr="category", 
                                            label_attr_converter=None, num_instance=num_instance))

dev = pd.concat(dev).reset_index(drop=True)

In [12]:
for p in dev[(dev["taskSize"] <= 5) & (dev["taskIndex"] == 1)].prompt:
    print(p)
    print("-"*50)
    print()

Classify which news category the following line of text belongs to among the following four categories: 'Business,' 'Sports,' 'World,' and 'Sci/Tech.'

Text: The last time they saw each other, John Kerry was sitting in the Red Sox owner #39;s box on the eve of the Democratic National Convention and the front 
News category:
--------------------------------------------------

Classify which news category each of the 5 following lines of text belongs to among the following four categories: 'Business,' 'Sports,' 'World,' and 'Sci/Tech.'

Texts, one per line:

1. Opec members call for a cut in rampant over-production as prices oil prices plummet amid warmer US weather.
2. Alex Rodriguez scores on a wild pitch in the 11th innning to lead the New York Yankees into the AL Championship Series by defeating the Minnesota Twins, 6-5, on Saturday.
3. The UK was today on course to become the second largest market for clothing retailer Hennes  amp; Mauritz after turnover climbed 16 over the summer.


In [13]:
dev["preds"] = dev.prompt.apply(get_completion)
dev

,taskIndex,prompt,answer,targetLabel,task,#shot,CoT,taskSize,preds
0,1,"Classify which news category the following line of text belongs to among the following four categories: 'Business,' 'Sports,' 'World,' and 'Sci/Te...",Sports,NA,single_clf,0,False,1,Sports
1,2,"Classify which news category the following line of text belongs to among the following four categories: 'Business,' 'Sports,' 'World,' and 'Sci/Te...",Sports,NA,single_clf,0,False,1,Sports
2,1,"Classify which news category each of the 5 following lines of text belongs to among the following four categories: 'Business,' 'Sports,' 'World,' ...","[World, Sports, Business, Business, World]",NA,batch_clf,0,False,5,1. Business\n2. Sports\n3. Business\n4. Business\n5. World
3,2,"Classify which news category each of the 5 following lines of text belongs to among the following four categories: 'Business,' 'Sports,' 'World,' ...","[World, Sports, Sports, Business, Sci/Tech]",NA,batch_clf,0,False,5,1. World\n2. Sports\n3. Sports\n4. Business\n5. Sci/Tech
4,1,"Classify which news category each of the 10 following lines of text belongs to among the following four categories: 'Business,' 'Sports,' 'World,'...","[World, Sports, Business, Business, World, Business, Business, Sci/Tech, Business, Sci/Tech]",NA,batch_clf,0,False,10,1. Business\n2. Sports\n3. Business\n4. Business\n5. World\n6. Sci/Tech\n7. Business\n8. Sci/Tech\n9. Business\n10. Sci/Tech
5,2,"Classify which news category each of the 10 following lines of text belongs to among the following four categories: 'Business,' 'Sports,' 'World,'...","[World, Sports, Sports, Business, Sci/Tech, Sci/Tech, World, Sports, Business, World]",NA,batch_clf,0,False,10,1. World\n2. Sports\n3. Sports\n4. Business\n5. Sci/Tech\n6. Sci/Tech\n7. World\n8. Sports\n9. Business\n10. World
6,1,"This is a news classification task in which each line of text belongs to one of four categories 'Business,' 'Sports,' 'World,' and 'Sci/Tech.'\n\n...","{3, 4}",Business,index_selection_one_cat_a_time,0,False,5,3.
7,1,"This is a news classification task in which each line of text belongs to one of four categories 'Business,' 'Sports,' 'World,' and 'Sci/Tech.'\n\n...",{None},Sci/Tech,index_selection_one_cat_a_time,0,False,5,None.
8,1,"This is a news classification task in which each line of text belongs to one of four categories 'Business,' 'Sports,' 'World,' and 'Sci/Tech.'\n\n...",{2},Sports,index_selection_one_cat_a_time,0,False,5,2.
9,1,"This is a news classification task in which each line of text belongs to one of four categories 'Business,' 'Sports,' 'World,' and 'Sci/Tech.'\n\n...","{1, 5}",World,index_selection_one_cat_a_time,0,False,5,5


### Adjusted

In [18]:
dev = []
num_instance = 2

taskSizes = [5, 10]
for propmtMode in ["zero-shot"]:
    for task in tasks[-1:]:
        if task == "single_clf":
            dev.append(make_prompts_for_clf(database, task, "dev", propmtMode)[:num_instance])
            continue

        for taskSize in taskSizes:  
            dev.append(make_prompts_for_clf(database, task, "dev", propmtMode, taskSize, attr="category", 
                                            label_attr_converter=None, num_instance=num_instance))

dev = pd.concat(dev).reset_index(drop=True)

In [20]:
for p in dev[(dev["taskSize"] <= 5) & (dev["taskIndex"] == 1)].prompt:
    print(p)
    print("-"*50)
    print()

This is a news classification task in which each line of text belongs to one of four categories 'Business,' 'Sports,' 'World,' and 'Sci/Tech.'

Go over the 5 lines of text below and list the index numbers of the lines that can be classified as business according to the following instructions:
If none of the texts can be classified as business, write 'None.'
If all the texts can be classified as business, write 'All.'
Otherwise, provide the index numbers of the texts that can be classified as business.

Output your responses in JSON format with the key 'business'.
A formatted example output is provided below.
 {'business': [None/All or index numbers of the texts that can be classified as business]}

Texts, one per line:

1. Opec members call for a cut in rampant over-production as prices oil prices plummet amid warmer US weather.
2. Alex Rodriguez scores on a wild pitch in the 11th innning to lead the New York Yankees into the AL Championship Series by defeating the Minnesota Twins, 6-5

In [21]:
model = "meta-llama/Llama-3-8b-chat-hf"
dev["preds"] = dev.prompt.apply(lambda p: get_completion(p, model))
dev

,taskIndex,prompt,answer,targetLabel,task,#shot,CoT,taskSize,preds
0,1,"This is a news classification task in which each line of text belongs to one of four categories 'Business,' 'Sports,' 'World,' and 'Sci/Tech.'\n\n...","{3, 4}",Business,index_selection_one_cat_a_time_json,0,False,5,"Here is the output in JSON format:\n\n{\n""business"": [1, 3, 4]\n}\n\nThe texts that can be classified as business are:\n\n* Line 1: Opec members c..."
1,1,"This is a news classification task in which each line of text belongs to one of four categories 'Business,' 'Sports,' 'World,' and 'Sci/Tech.'\n\n...",{None},Sci/Tech,index_selection_one_cat_a_time_json,0,False,5,"Here is the output in JSON format:\n\n{\n""sci/tech"": [4, 5]\n}\n\nThe texts that can be classified as sci/tech are:\n\n* Line 4: ""Blockbuster Inc...."
2,1,"This is a news classification task in which each line of text belongs to one of four categories 'Business,' 'Sports,' 'World,' and 'Sci/Tech.'\n\n...",{2},Sports,index_selection_one_cat_a_time_json,0,False,5,"Here is the output in JSON format:\n\n{\n""sports"": [2]\n}\n\nThe text at index 2 can be classified as sports, as it mentions a baseball game and s..."
3,1,"This is a news classification task in which each line of text belongs to one of four categories 'Business,' 'Sports,' 'World,' and 'Sci/Tech.'\n\n...","{1, 5}",World,index_selection_one_cat_a_time_json,0,False,5,"Here is the output in JSON format:\n\n{\n""world"": [5]\n}\n\nThe text at index 5 can be classified as ""World""."
4,2,"This is a news classification task in which each line of text belongs to one of four categories 'Business,' 'Sports,' 'World,' and 'Sci/Tech.'\n\n...",{4},Business,index_selection_one_cat_a_time_json,0,False,5,"Here is the output in JSON format:\n\n{\n""business"": [4]\n}\n\nThe text at index 4 can be classified as business, as it discusses British economic..."
5,2,"This is a news classification task in which each line of text belongs to one of four categories 'Business,' 'Sports,' 'World,' and 'Sci/Tech.'\n\n...",{5},Sci/Tech,index_selection_one_cat_a_time_json,0,False,5,"Here is the output in JSON format:\n\n{\n""sci/tech"": [4, 5]\n}\n\nThe lines that can be classified as sci/tech are:\n\n* Line 4: British economic ..."
6,2,"This is a news classification task in which each line of text belongs to one of four categories 'Business,' 'Sports,' 'World,' and 'Sci/Tech.'\n\n...","{2, 3}",Sports,index_selection_one_cat_a_time_json,0,False,5,"Here is the JSON output:\n\n{\n""sports"": [2, 3]\n}\n\nThe lines that can be classified as sports are:\n\n* Line 2: ""Listening intently as the nati..."
7,2,"This is a news classification task in which each line of text belongs to one of four categories 'Business,' 'Sports,' 'World,' and 'Sci/Tech.'\n\n...",{1},World,index_selection_one_cat_a_time_json,0,False,5,"Here is the output in JSON format:\n\n{\n""world"": [4]\n}\n\nThe text at index 4 can be classified as ""World"" because it discusses British economic..."
8,1,"This is a news classification task in which each line of text belongs to one of four categories 'Business,' 'Sports,' 'World,' and 'Sci/Tech.'\n\n...","{3, 4, 6, 7, 9}",Business,index_selection_one_cat_a_time_json,0,False,10,"Here is the JSON output:\n\n{\n""business"": [1, 3, 4, 6, 7, 9, 10]\n}\n\nThe lines that can be classified as business are:\n\n* Line 1: Opec member..."
9,1,"This is a news classification task in which each line of text belongs to one of four categories 'Business,' 'Sports,' 'World,' and 'Sci/Tech.'\n\n...","{8, 10}",Sci/Tech,index_selection_one_cat_a_time_json,0,False,10,"Here is the JSON output:\n\n{\n""sci/tech"": [4, 6, 8]\n}\n\nThe lines that can be classified as sci/tech are:\n\n* Line 4: Blockbuster Inc., the wo..."


### Make prompts

In [45]:
database = read_obj_from_pickle(f"data/databases/text classification/{clf_task}.pkl")

Read object from data/databases/text classification/AGNews.pkl


In [15]:
# out = []
# num_instance = 100

# taskSizes = [5, 10, 20, 50, 100]
# for propmtMode in ["zero-shot"]:
#     for task in tasks:

#         if task == "single_clf":
#             out.append(make_prompts_for_clf(database, task, "test", propmtMode))
#             continue

#         for taskSize in taskSizes:
#             out.append(make_prompts_for_clf(database, task, "test", propmtMode, taskSize, attr="category", 
#                                             label_attr_converter=None, num_instance=num_instance))

# out = pd.concat(out)
# out.reset_index(drop=True, inplace=True)

In [16]:
# os.makedirs("results/text classification/", exist_ok=True)
# out.to_json(f"results/text classification/{clf_task}.json", orient="records", lines=True)                                             

In [52]:
fp = f"results/text classification/{clf_task}.json"
out = [pd.read_json(fp, lines=True)]

num_instance = 100
taskSizes = [5, 10, 20, 50, 100]
for propmtMode in ["zero-shot"]:
    for taskSize in taskSizes:
        out.append(make_prompts_for_clf(database, tasks[-1], "test", propmtMode, taskSize, attr="category", 
                                        label_attr_converter=None, num_instance=num_instance))

out = pd.concat(out)
out.reset_index(drop=True, inplace=True)

os.makedirs("results/text classification/", exist_ok=True)
out.to_json(f"results/text classification/{clf_task}.json", orient="records", lines=True)                                             

In [53]:
out.task.value_counts()

index_selection_one_cat_a_time         2000
index_selection_one_cat_a_time_json    2000
single_clf                             1000
batch_clf                               500
index_selection_all_cat_at_once         500
Name: task, dtype: int64

In [54]:
out.prompt.apply(get_num_of_tokens).describe()

count    6000.000000
mean     1172.116500
std      1184.802133
min        47.000000
25%       324.000000
50%       629.500000
75%      1827.000000
max      3706.000000
Name: prompt, dtype: float64

In [29]:
sub = out.copy()[(out.taskIndex == 1) & (out.taskSize <= 5)]
sub

,taskIndex,prompt,answer,targetLabel,task,#shot,CoT,taskSize,gpt-3.5-turbo-0125-completion,meta-llama/Llama-3-70b-chat-hf-completion,mistralai/Mixtral-8x7B-Instruct-v0.1-completion,meta-llama/Llama-3-8b-chat-hf-completion,gpt-4-turbo-2024-04-09-completion,lmsys/vicuna-13b-v1.5-completion,mistralai/Mistral-7B-Instruct-v0.2-completion
0,1,"Classify which news category the following line of text belongs to among the following four categories: 'Business,' 'Sports,' 'World,' and 'Sci/Te...",Sports,NA,single_clf,0,False,1,Sports,The news category is: 'Sports.',"Sports) The text falls into the 'Sports' category as it is about a professional American football team, the New York Jets, and their quarterback,...",The correct answer is: 'Sports',"The news category for the text ""The New York Jets and quarterback Chad Pennington are looking to finalize a contract extension by next Wednesday.""...",Sports,Sports.
1000,1,"Classify which news category each of the 5 following lines of text belongs to among the following four categories: 'Business,' 'Sports,' 'World,' ...","[Business, Business, Sci/Tech, Business, Sports]",NA,batch_clf,0,False,5,1. Business\n2. Business\n3. Sci/Tech\n4. World\n5. Sports,Here are the classifications:\n\n1. Business\n2. Business\n3. Sci/Tech\n4. World\n5. Sports,Business\nBusiness\nSci/Tech\nWorld\nSports,Here are the classifications:\n\n1. Business\n2. Business\n3. Sci/Tech\n4. World\n5. Sports,1. Business\n2. Business\n3. Sci/Tech\n4. World\n5. Sports,1. Business\n2. Sci/Tech\n3. Business\n4. World\n5. Sports,1. Business: New Zealand's High Court rejected a proposed alliance between Air New Zealand Ltd. and Australia's Qantas Airways Ltd.\n2. Business:...
1500,1,"This is a news classification task in which each line of text belongs to one of four categories 'Business,' 'Sports,' 'World,' and 'Sci/Tech.'\n\n...","[1, 2, 4]",Business,index_selection_one_cat_a_time,0,False,5,1\n2,1\n2\n3,"1\n2\n3\nNone of the other categories seem to apply to these texts, so I'll go with the index numbers that can be classified as business.",Here is the classification:\n\n1\n2\n3\n\nThese three lines can be classified as business.,1\n2,None.,"1. The first line can be classified as Business: The text mentions two airlines, Air New Zealand and Qantas, which are companies in the Business ..."
1501,1,"This is a news classification task in which each line of text belongs to one of four categories 'Business,' 'Sports,' 'World,' and 'Sci/Tech.'\n\n...",[3],Sci/Tech,index_selection_one_cat_a_time,0,False,5,2\n3,3\n4,"3\nThe third line can be classified as sci/tech because it mentions a specific technology product, Apple Computer's rack-mounted storage system, ...",2\n3,3,None.,3.\n\nExplanation:\nText 1 is about business and economics.\nText 2 is about business and world news.\nText 4 is about infrastructure and politic...
1502,1,"This is a news classification task in which each line of text belongs to one of four categories 'Business,' 'Sports,' 'World,' and 'Sci/Tech.'\n\n...",[5],Sports,index_selection_one_cat_a_time,0,False,5,None.,5,"5\n\nThe fifth text can be classified as sports because it refers to ""Brian Kerr took his Ireland squad to the dogs last Wednesday,"" which implie...",5,5,None.,"Index number 5 can be classified as sports. Therefore, the answer is:\n\n5.\n\nExplanation: The fifth text mentions Brian Kerr taking his Ireland..."
1503,1,"This is a news classification task in which each line of text belongs to one of four categories 'Business,' 'Sports,' 'World,' and 'Sci/Tech.'\n\n...",[None],World,index_selection_one_cat_a_time,0,False,5,2.,1\n2\n4,"2\n5\nHere's the reasoning:\n1. This text is about an event in New Zealand and Australia, so it doesn't fit the 'World' category.\n2. This text i...",1\n2,1\n2,None.,"2.\n\nExplanation:\nText 1 is about business and mergers between two airlines, not related to the world category.\nText 3 is about Apple Computer..."
3500,1,"This is a news classification task in which each line 

In [56]:
out["lmsys/vicuna-13b-v1.5-completion"].str.contains("TOO_LONG_SKIP").sum()

600

In [57]:
for ix in out[out.taskSize == 100].index:
    out.at[ix, "lmsys/vicuna-13b-v1.5-completion"] = "TOO_LONG_SKIP"

In [58]:
out["lmsys/vicuna-13b-v1.5-completion"].str.contains("TOO_LONG_SKIP").sum()

1000

In [59]:
out.to_json(f"results/text classification/{clf_task}.json", orient="records", lines=True)